In [36]:
import pandas as pd
import numpy as np

df = pd.read_csv('energydata_complete.csv')
#df.info()
df.date = pd.to_datetime(df['date'])
print(df.dtypes)
#df.head(5)

date           datetime64[ns]
Appliances              int64
lights                  int64
T1                    float64
RH_1                  float64
T2                    float64
RH_2                  float64
T3                    float64
RH_3                  float64
T4                    float64
RH_4                  float64
T5                    float64
RH_5                  float64
T6                    float64
RH_6                  float64
T7                    float64
RH_7                  float64
T8                    float64
RH_8                  float64
T9                    float64
RH_9                  float64
T_out                 float64
Press_mm_hg           float64
RH_out                float64
Windspeed             float64
Visibility            float64
Tdewpoint             float64
rv1                   float64
rv2                   float64
dtype: object


In [37]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df.drop(columns='date')), columns=df.columns[1:])
feature = np.array(normalised_df['T2']).reshape(-1,1)
target = normalised_df['T6']

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=1)

linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

predicted_values = linear_model.predict(x_test)
predicted_values[:4]

array([0.27457749, 0.44635267, 0.59807124, 0.26228473])

In [41]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2) 

0.65

In [52]:
df13 = df.drop(columns=['date', 'lights'])
normalised_df = pd.DataFrame(scaler.fit_transform(df13), columns=df13.columns)
#normalised_df.columns

feature = df13.drop(columns='Appliances')
target = df13.Appliances

x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=42)

linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

predicted_values = linear_model.predict(x_test)
display(predicted_values[:4])

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2) 

array([ 45.54761868, 271.20410666,  46.38025696, 122.75338191])

53.64

In [53]:
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2) 

51918501.21

In [54]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

93.64

In [58]:
def get_weights_df(model, feat, col_name):
    """this function returns the weight of every feature
    """
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_model_weight')
display(linear_model_weights)

,Features,Linear_model_weight
0,T9,-21.148452
1,T2,-18.370030
2,RH_2,-13.740748
3,T_out,-11.073628
4,RH_8,-5.778861
5,RH_9,-1.762680
6,RH_7,-1.692787
7,T5,-1.600843
8,RH_out,-1.093521
9,T1,-0.370721


In [60]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

ridge_predicted_values = ridge_reg.predict(x_test)
#ridge_predicted_values[:4]

rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

93.64

In [64]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

lasso_predicted_values = lasso_reg.predict(x_test)
#lasso_predicted_values[:4]

lasso_model_weight = get_weights_df(lasso_reg, x_train, 'lasso_model_weight')
display(len(lasso_model_weight!=0))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6136226.882201627, tolerance: 14642.917148544962
  model = cd_fast.enet_coordinate_descent(


26

In [65]:
rmse = np.sqrt(mean_squared_error(y_test, lasso_predicted_values))
round(rmse, 3)

93.64